In [1]:
# import relevant libraries
import pandas as pd
import numpy as np

In [2]:
# load the csvs
df1 = pd.read_csv('../ScrapeData/csv_recipes/recipe_df1.csv')
df2 = pd.read_csv('../ScrapeData/csv_recipes/recipe_df2.csv')
df3 = pd.read_csv('../ScrapeData/csv_recipes/recipe_df3.csv')
df4 = pd.read_csv('../ScrapeData/csv_recipes/recipe_df4.csv')
df5 = pd.read_csv('../ScrapeData/csv_recipes/recipe_df5.csv')

In [3]:
# concat the dataframes
df = pd.concat([df1,df2,df3,df4,df5])
df = df.drop(['Unnamed: 0', 'id'], axis=1)

In [4]:
df.head()

,bestrating,imagesrc,ingredients,makeitagainscore,preparation,rating,reviews,tags,title
0,4.0,https://assets.epicurious.com/photos/56df1d8cb...,"['2 cups whole-wheat flour', '2 cups all-purpo...",90%,Put oven rack in middle position and preheat o...,3.5,50.0,"['Irish', 'Bread', 'Bake', 'Vegetarian', 'Quic...",Irish Brown Bread
1,4.0,https://assets.epicurious.com/photos/5a78b5919...,"['1 Tbsp. crushed red pepper flakes', '1 Tbsp....",91%,"Coarsely grind red pepper flakes, peppercorns,...",3.0,11.0,"['Pepper', 'Fennel', 'Pork', 'Garlic', 'White ...",Slow-Cooked Pork with Chickpeas
2,4.0,https://assets.epicurious.com/photos/5a68f33b3...,"['1/4 cup extra-virgin olive oil', '12 oz. dry...",67%,Pour oil into cooker insert and select “Sauté....,2.0,15.0,"['Instant Pot', 'Kale', 'Potato', 'Onion', 'Ga...",Instant Pot Caldo Verde
3,4.0,https://assets.epicurious.com/photos/5a9971b01...,"['1 tablespoon olive oil', '2 garlic cloves, t...",81%,Preheat oven to 350°F. Heat oil in a large ski...,3.5,26.0,"['Side', 'Bake', 'Christmas', 'Easter', 'Thank...",Parmesan Bread Pudding with Broccoli Rabe and ...
4,4.0,https://assets.epicurious.com/photos/54b71d9a2...,"['1 1/2 pounds boneless, skinless chicken brea...",96%,"Place chicken, garlic, bay leaves, allspice, a...",3.5,23.0,"['Asian', 'Soup/Stew', 'Chicken', 'Low Carb', ...",Brothy Poached Chicken with Mushrooms and Fres...


In [5]:
def classify_rating(rating):
    if rating >= 4:
        return 'good'
    else:
        return 'bad'

In [6]:
df['class'] = df['rating'].apply(classify_rating)

In [7]:
df.head(10)

,bestrating,imagesrc,ingredients,makeitagainscore,preparation,rating,reviews,tags,title,class
0,4.0,https://assets.epicurious.com/photos/56df1d8cb...,"['2 cups whole-wheat flour', '2 cups all-purpo...",90%,Put oven rack in middle position and preheat o...,3.5,50.0,"['Irish', 'Bread', 'Bake', 'Vegetarian', 'Quic...",Irish Brown Bread,bad
1,4.0,https://assets.epicurious.com/photos/5a78b5919...,"['1 Tbsp. crushed red pepper flakes', '1 Tbsp....",91%,"Coarsely grind red pepper flakes, peppercorns,...",3.0,11.0,"['Pepper', 'Fennel', 'Pork', 'Garlic', 'White ...",Slow-Cooked Pork with Chickpeas,bad
2,4.0,https://assets.epicurious.com/photos/5a68f33b3...,"['1/4 cup extra-virgin olive oil', '12 oz. dry...",67%,Pour oil into cooker insert and select “Sauté....,2.0,15.0,"['Instant Pot', 'Kale', 'Potato', 'Onion', 'Ga...",Instant Pot Caldo Verde,bad
3,4.0,https://assets.epicurious.com/photos/5a9971b01...,"['1 tablespoon olive oil', '2 garlic cloves, t...",81%,Preheat oven to 350°F. Heat oil in a large ski...,3.5,26.0,"['Side', 'Bake', 'Christmas', 'Easter', 'Thank...",Parmesan Bread Pudding with Broccoli Rabe and ...,bad
4,4.0,https://assets.epicurious.com/photos/54b71d9a2...,"['1 1/2 pounds boneless, skinless chicken brea...",96%,"Place chicken, garlic, bay leaves, allspice, a...",3.5,23.0,"['Asian', 'Soup/Stew', 'Chicken', 'Low Carb', ...",Brothy Poached Chicken with Mushrooms and Fres...,bad
5,4.0,https://assets.epicurious.com/photos/5a32af154...,"['6 tablespoons extra-virgin olive oil, divide...",100%,Heat 4 Tbsp. oil in a large deep-sided skillet...,3.5,27.0,"['Quick and Healthy', 'Healthy', 'Pasta', 'Veg...","Lemony Pasta with Cauliflower, Chickpeas, and ...",bad
6,4.0,https://assets.epicurious.com/photos/5a270eca0...,"['1 (10-pound) smoked ham with rind, preferabl...",93%,Preheat oven to 325°F. Place ham in large roas...,4.0,47.0,"['Fruit Juice', 'Mustard', 'Bake', 'Christmas'...",Old-Fashioned Ham with Brown Sugar and Mustard...,good
7,4.0,https://assets.epicurious.com/photos/5a3002b75...,"['1 cup polenta (not quick-cooking)', '1 teasp...",90%,Whisk polenta and 4 cups water in pressure coo...,2.5,10.0,"['Bon Appétit', 'Quick & Easy', 'Dinner', 'Cor...",Polenta Cacio e Pepe,bad
8,4.0,https://assets.epicurious.com/photos/5a3002b53...,['2 tablespoons virgin coconut oil or extra-vi...,100%,Heat oil in a medium saucepan over medium. Coo...,4.0,40.0,"['Bon Appétit', 'Soup/Stew', 'Winter', 'Lentil...","Curried Lentil, Tomato, and Coconut Soup",good
9,4.0,https://assets.epicurious.com/photos/5a2ac2b14...,"['7 slices white sandwich bread', '1 pound bul...",100%,Preheat oven to 425°F. Toast bread on a rimmed...,4.0,15.0,"['Breakfast', 'Sausage', 'Egg', 'Yogurt', 'Gre...",Cowboy Breakfast Casserole with Sausage and Sp...,good


In [8]:
# create has_image feature
df['has_image'] = np.where(df['imagesrc'].isnull(), 'No', 'Yes')

In [9]:
df = df.dropna(axis=0, how='any', subset=['preparation','title'])

In [27]:
df['class'].value_counts()

bad     5474
good    1198
Name: class, dtype: int64

### Train test split

In [10]:
#encode labels to integer classes
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder().fit(df['class'])
lb.classes_
lb.transform(df['class'])

array([0, 0, 0, ..., 1, 0, 0])

In [31]:
#null model(baseline)
1-lb.transform(df['class']).mean()

0.82044364508393286

In [11]:
from sklearn.model_selection import train_test_split

X_train_part, X_valid, y_train_part, y_valid =\
    train_test_split(df, 
                     lb.transform(df['class']), 
                test_size=0.3,random_state=17, stratify=df['class'])

In [12]:
print(X_train_part.shape)
print(X_valid.shape)
print(y_train_part.shape)
print(y_valid.shape)

(4670, 11)
(2002, 11)
(4670,)
(2002,)


### Preparing a pipeline

In [14]:
from sklearn.base import BaseEstimator, TransformerMixin

class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key.

    The data is expected to be stored in a 2D data structure, where the first
    index is over features and the second is over samples.  i.e.

    >> len(data[key]) == n_samples

    Please note that this is the opposite convention to scikit-learn feature
    matrixes (where the first index corresponds to sample).

    ItemSelector only requires that the collection implement getitem
    (data[key]).  Examples include: a dict of lists, 2D numpy array, Pandas
    DataFrame, numpy record array, etc.

    >> data = {'a': [1, 5, 2, 5, 2, 8],
               'b': [9, 4, 1, 4, 1, 3]}
    >> ds = ItemSelector(key='a')
    >> data['a'] == ds.transform(data)

    ItemSelector is not designed to handle data grouped by sample.  (e.g. a
    list of dicts).  If your data is structured this way, consider a
    transformer along the lines of `sklearn.feature_extraction.DictVectorizer`.

    Parameters
    ----------
    key : hashable, required
        The key corresponding to the desired value in a mappable.
    """
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]

In [20]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer

pipeline = Pipeline([
  
  ('union', FeatureUnion([
    ('tf_idf_preparation', Pipeline([
      ('selector', ItemSelector(key='preparation')),
      ('counts', CountVectorizer()),
      ('tf_idf', TfidfTransformer())
    ]))
  ])),
  ('classifier', RandomForestClassifier())
])

In [21]:
pipeline.fit(X_train_part, y_train_part)

Pipeline(memory=None,
     steps=[('union', FeatureUnion(n_jobs=1,
       transformer_list=[('tf_idf_preparation', Pipeline(memory=None,
     steps=[('selector', ItemSelector(key='preparation')), ('counts', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [22]:
pipeline.steps

[('union', FeatureUnion(n_jobs=1,
         transformer_list=[('tf_idf_preparation', Pipeline(memory=None,
       steps=[('selector', ItemSelector(key='preparation')), ('counts', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=None, min_df=1,
          ngram_range=(1, 1), ...=None)), ('tf_idf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True))]))],
         transformer_weights=None)),
 ('classifier',
  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
              max_depth=None, max_features='auto', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
              oob_score=False, random_state=None, verbose=0,
              war

In [24]:
from sklearn import metrics
pred = pipeline.predict(X_valid)
metrics.accuracy_score(y_valid, pred)

0.82117882117882113